In [14]:
import torch
import numpy as np

torch.cuda.is_available()

True

### Tensors

Tensors are similar to NumPy’s ndarrays, except that tensors can run on GPUs or other hardware accelerators. In fact, tensors and NumPy arrays can often share the same underlying memory, eliminating the need to copy data (see Bridge with NumPy). Tensors are also optimized for automatic differentiation 

In [15]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
x_data


tensor([[1, 2],
        [3, 4]])

In [16]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [17]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")
x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.9106, 0.0301],
        [0.8134, 0.4649]]) 



In [18]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


### Operations on Tensors

Over 1200 tensor operations, including arithmetic, linear algebra, matrix manipulation (transposing, indexing, slicing), sampling and more are comprehensively described here.

Each of these operations can be run on the CPU and Accelerator such as CUDA, MPS, MTIA, or XPU. If you’re using Colab, allocate an accelerator by going to Runtime > Change runtime type > GPU.

By default, tensors are created on the CPU. We need to explicitly move tensors to the accelerator using .to method (after checking for accelerator availability). Keep in mind that copying large tensors across devices can be expensive in terms of time and memory!

In [19]:
# We move our tensor to the current accelerator if available
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

### Arithmetic operations

Let $A$ be a matrix of size $m \times n$.

Then:

-   $A^T$ is size $n \times m$
    
-   So, two possible multiplications:
    
    1.  $A^T A$ — **results in a square matrix of size $n \times n$**
        
    2.  $A A^T$ — **results in a square matrix of size $m \times m$**
        

---

### 🧠 What does it *mean*?

#### 🔷 1. $A^T A$: Inner product / Gram matrix

-   Each entry is:
    
    $$
    (A^T A)_{ij} = \langle \text{col}_i, \text{col}_j \rangle
    $$
    
-   It represents **dot products between columns** of $A$.
    
-   The result is **symmetric** and **positive semi-definite**.
    
-   Common in **least squares**, **PCA**, **SVD**, and **machine learning** kernels.
    

#### 🔷 2. $A A^T$: Outer product form

-   Each entry is:
    
    $$
    (A A^T)_{ij} = \langle \text{row}_i, \text{row}_j \rangle
    $$
    
-   Represents **dot products between rows** of $A$.
    
-   Also **symmetric** and **positive semi-definite**.
    
-   Shows up in **covariance matrices**, **projections**, etc.
    

---

### 🧪 Example:

Let:

$$
A = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix}
$$

Then:

**Transpose:**

$$
A^T = \begin{bmatrix} 1 & 3 \\ 2 & 4 \\ \end{bmatrix}
$$

**Multiply:**

$$
A^T A = \begin{bmatrix} 1 & 3 \\ 2 & 4 \\ \end{bmatrix} \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} = \begin{bmatrix} 10 & 14 \\ 14 & 20 \\ \end{bmatrix}
$$

**Notice**:

-   $A^T A$ is square, symmetric.
    
-   $A A^T$ would be $2 \times 2$ as well, and also symmetric.
    

---

### 🧭 Geometrically:

-   $A^T A$ gives you a **Gram matrix** that captures **angles** and **lengths** between the column vectors — essential in measuring **linear dependence**.
    
-   If columns are orthonormal ⇒ $A^T A = I$
    

---

### 🚀 Applications:

-   **Machine Learning**: $X^T X$ in linear regression.
    
-   **PCA**: Eigenvalues of $A^T A$ give variance directions.
    
-   **Signal Processing**: Autocorrelation.
    
-   **Optimization**: Hessians and quadratic forms.
    

---

So, multiplying a matrix with its transpose is like asking:

> "How do the rows or columns of this matrix relate to each other in space?"

It exposes symmetry, structure, and often, the soul of the data.

In [20]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[0.1149, 0.8537, 0.6894, 0.9071],
        [0.0180, 0.7565, 0.0609, 0.2186],
        [0.7478, 0.0685, 0.8693, 0.6612]], device='cuda:0')

### Bridge with NumPy

Tensors on the CPU and NumPy arrays can share their underlying memory locations, and changing one will change the other.

In [21]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")


t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [22]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [23]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [25]:
from torch import nn


class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10)
		)

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [26]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


### Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [27]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [28]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [29]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [30]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4665, -0.3476,  0.0528,  0.0341, -0.1381, -0.6701,  0.2596, -0.0893,
          0.1842,  0.0037,  0.1799,  0.7648, -0.1874, -0.2829,  0.3908,  0.4948,
          0.0124, -0.0305,  0.1092,  0.4039],
        [-0.3238, -0.3212,  0.0972, -0.1738, -0.4147, -0.6638,  0.1154, -0.1260,
          0.3321, -0.2809,  0.0849,  0.5474,  0.1532, -0.3480,  0.6300,  0.4765,
         -0.2374, -0.0717, -0.2039, -0.0507],
        [-0.4527, -0.2340,  0.2043,  0.1281, -0.0867, -0.2764, -0.2231, -0.4015,
          0.2992, -0.0422, -0.0138,  0.7040,  0.2096, -0.7557,  0.9217,  0.2168,
          0.0439,  0.1405, -0.0802, -0.1206]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0528, 0.0341, 0.0000, 0.0000, 0.2596, 0.0000, 0.1842,
         0.0037, 0.1799, 0.7648, 0.0000, 0.0000, 0.3908, 0.4948, 0.0124, 0.0000,
         0.1092, 0.4039],
        [0.0000, 0.0000, 0.0972, 0.0000, 0.0000, 0.0000, 0.1154, 0.0000, 0.3321,
         0.0000, 0.0849, 0.5474, 0.1532, 0.0000, 0.63

## nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [32]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1009, 0.1086, 0.0970, 0.1029, 0.1009, 0.0933, 0.1058, 0.0949, 0.1058,
         0.0899]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

## Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [33]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0320, -0.0269,  0.0155,  ..., -0.0074,  0.0117, -0.0110],
        [ 0.0019, -0.0084, -0.0140,  ...,  0.0065, -0.0048,  0.0286]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0225, 0.0244], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0057, -0.0141, -0.0111,  ..., -0.0381,  0.0251, -0.0294],
        [ 0.0102,  0.0093,  0.0409,  ...,  0.0082, -0.0003, -0.0075]],
       device='cuda:0', grad_fn=<Slic